<a href="https://colab.research.google.com/github/GTWFam/Group7_DS3010/blob/main/CaseStudy2_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study 2 : Putting Twitter Data into the Cloud

Due Date: 2/15/2020, **BEFORE the beginning of class at 2:00pm EST**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

<a title="Fabián Alexis [CC BY-SA (https://creativecommons.org/licenses/by-sa/3.0)]" href="https://commons.wikimedia.org/wiki/File:Antu_mongodb.svg"><img width="50" alt="Antu mongodb" src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/Antu_mongodb.svg/512px-Antu_mongodb.svg.png"></a>

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    Jacob Bissonette
    Austin Franklin 
    Yuriy Kamenivskyy 
    Garrett McMerriman
    Bill Hoang Pham

**Don't forget!**
* You will need to install the pymongo library to access MongoDB
 * pip install pymongo
* You will also need to install dnspython to access the cloud version of MongoDB
 * pip install dnspython

** NOTE **
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.

# Problem 1 (20 points):  Get a cloud database account using MongoDB Atlas

Get a free cloud hosted MongoDB database account at https://www.mongodb.com/atlas/database.  You will need to: 

* Create a database cluster using the MongoDB web interface
* Get your MongoDB credentials
* Create a document collection in MongoBD


# Problem 2 (20 points):  Read Twitter data into your cloud MongoDB database

Gather Twitter data and upload it to your cloud database.  You will need to:

* Get your cloubd MongoDB credentials using the MongoDB web interface
* Insert those credentials into your Juypyter notebook using pymongo
* Gather Twitter data as in Case Study 1
* Upload your Twitter data to the MongoDB cloud document collection

Note, to do you this question remember that you will need *two sets of credentials*

* One set for the MongoDB Atlas database
* One set for the Twitter developed API

Of course, there are not the same!

In [1]:
! pip install pymongo
! pip install dnspython


     |████████████████████████████████| 266 kB 4.3 MB/s 


In [2]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import pymongo
import json

from google.colab import drive

drive.mount('/content/drive')

data = {}
    
# Extracting the saved credentials from the drive
with open('drive/MyDrive/ds3010_2022/credentials.txt', 'r') as f:
    data = json.load(f)
CONSUMER_KEY = data["consumer_key"]
CONSUMER_SECRET = data["consumer_secret"]
OAUTH_TOKEN = data["oauth_token"]
OAUTH_TOKEN_SECRET = data["oauth_token_secret"]
mongodb_url = data["mongodb_url"]

client = pymongo.MongoClient(mongodb_url)


Mounted at /content/drive


In [3]:
# Getting the Database and Collection from MongoDB
db = client['DS3010']
coll = db['CS2']

In [4]:
! pip install twitter

     |████████████████████████████████| 50 kB 2.5 MB/s 


In [5]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import twitter

# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation.

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [6]:
query = "macbook pro 2021"

data = twitter_api.search.tweets(q=query, lang='en', count=100)

min_id = data['statuses'][-1]['id']
for i in range(7):
    tweets = twitter_api.search.tweets(q=query, lang='en', max_id=min_id, count=100)
    min_id = tweets['statuses'][-1]['id']
    data['statuses'] += tweets['statuses']


In [7]:
statuses = data['statuses']
# coll.drop()
# _ = coll.insert_many(statuses)

# Problem 3 (20 points):  Use a regular expression to read a subset of your Tweets out of MongoDB

Use regular expressions to analyze properties of your data.  For example, you can 

* search for trends that have a large volume, 
* search for tweets with geotags, 
* search for tweets that can contain a certain string,
* anything else you think might be useful for a product!

In [8]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

query = { "favorite_count": { "$gt": 20 } }
cursor = coll.find(query)

for tweet in cursor:
    print(tweet['text'])

Apple Says Pro Display XDR and 2021 MacBook Pro Can Experience Limited Brightness in High Temperatures by @julipuli… https://t.co/gOTRx0lgA9
2021 14 inch MacBook Pro 
M1 pro 16gb 512SSD 
GH₵ 14,399.95

iPhone 11 128gb Factory Unlocked GH₵ 3,999.95

☎️055-3… https://t.co/DDfpSEc47Q
Legitimately one of the best feelings. I just bought this 2021 Macbook Pro and paid for it entirely with NFT profit… https://t.co/fyFxSQ3DTX


In [9]:
query2 = { "geo": { "$ne": None } }
cursor = coll.find(query2)

for tweet in cursor:
    print(tweet['text'])

Two weeks used 
Battery Cycle Count 12 
2021 MacBook Pro 
14 inch 16 GB 512 SSD 
Box with accessories
Cool offer @… https://t.co/qtzSPqoWtR


In [10]:
query3 = { "text": { '$regex': '^.*cool.*$', '$options': 'si' } }
cursor = coll.find(query3)

for tweet in cursor:
    print(tweet['text'])

RT @AWereChicken: To help get closer to my surgery goal, I am willing to sell my MacBook Pro 2021 with touchbar, charger, and cool Ouija sk…
RT @AWereChicken: To help get closer to my surgery goal, I am willing to sell my MacBook Pro 2021 with touchbar, charger, and cool Ouija sk…
RT @AWereChicken: To help get closer to my surgery goal, I am willing to sell my MacBook Pro 2021 with touchbar, charger, and cool Ouija sk…
RT @AWereChicken: To help get closer to my surgery goal, I am willing to sell my MacBook Pro 2021 with touchbar, charger, and cool Ouija sk…
RT @AWereChicken: To help get closer to my surgery goal, I am willing to sell my MacBook Pro 2021 with touchbar, charger, and cool Ouija sk…
RT @AWereChicken: To help get closer to my surgery goal, I am willing to sell my MacBook Pro 2021 with touchbar, charger, and cool Ouija sk…
To help get closer to my surgery goal, I am willing to sell my MacBook Pro 2021 with touchbar, charger, and cool Ou… https://t.co/6aJFYR2wl4
Two weeks use

In [11]:
query4 = { "text": { '$regex': '^.*battery.*$', '$options': 'si' } }
cursor = coll.find(query4)

for tweet in cursor:
    print(tweet['text'])

Apple’s 2021 MacBook Pro Lasts 60% Longer Than the Best High-End Notebook’s Battery Timing, Claims New Test https://t.co/q6gxK4gRQ0
RT @wccftech: Apple’s 2021 MacBook Pro Lasts 60% Longer Than the Best High-End Notebook’s Battery Timing, Claims New Test https://t.co/NFvB…
Apple’s 2021 MacBook Pro Lasts 60% Longer Than the Best High-End Notebook’s Battery Timing, Claims New Test https://t.co/AQlBlbz7rj
Apple’s 2021 MacBook Pro Lasts 60% Longer Than the Best High-End Notebook’s Battery Timing, Claims New Test… https://t.co/WhbFZLMDud
Design and battery life favour the 2021 edition of the Microsoft #SurfaceProX but the Apple Macbook Pro and Air may… https://t.co/4wJ2VPFauQ
RT @Mint_Lounge: Design and battery life favour the 2021 edition of the Microsoft #SurfaceProX but the Apple Macbook Pro and Air may prove…
RT @Mint_Lounge: Design and battery life favour the 2021 edition of the Microsoft #SurfaceProX but the Apple Macbook Pro and Air may prove…
Two weeks used 
Battery Cycle Count 12 
2

# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the MongoDB.

* Come up with a business question that Twitter data and MongoDB could help answer.
* Decribe the business case.
* How could MongoDB help a company to scale up its computation environment?

Our business model is looking at customer's sentiment for any subject. For instance, we looked at people's attitudes for burgers to help the local resturants to select best burger menu items.
We collected both negative attitude tweets and positive attitude tweets.

MongoDB helps us organize the data into different collection and search with regular expressions faster in the whole database.

In [131]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

# Negative attitudes
query1 = 'burger OR burgers :( -filter:retweets'

data1 = twitter_api.search.tweets(q=query1, lang='en', count=100)

min_id = data1['statuses'][-1]['id']
for i in range(7):
    tweets = twitter_api.search.tweets(q=query1, lang='en', max_id=min_id, count=100)
    min_id = tweets['statuses'][-1]['id']
    data1['statuses'] += tweets['statuses']

statuses1 = data1['statuses']
print(len(statuses1))
coll_neg = db['cs2_coll_neg']
coll_neg.drop()
_ = coll_neg.insert_many(statuses1)

In [139]:
# Positive attitudes
query2 = 'burger OR burgers :) -filter:retweets'

data2 = twitter_api.search.tweets(q=query2, lang='en', count=100)

min_id = data2['statuses'][-1]['id']
for i in range(8):
    tweets = twitter_api.search.tweets(q=query2, lang='en', max_id=min_id, count=100)
    min_id = tweets['statuses'][-1]['id']
    data2['statuses'] += tweets['statuses']

statuses2 = data2['statuses']
print(len(statuses2))
coll_pos = db['cs2_coll_pos']
coll_pos.drop()
_ = coll_pos.insert_many(statuses2)

In [154]:
# Similar to Case Study 1, looking at most common words

import re
exclude = ['rt', 'in', 'the', 'for','of', 'to', 'and', 'a', 'is', 'are', 'on', 'from', 'that', 'this', 'these', 'with', 'what', 'it', 'if', 'or', 'but', 'your', 'be', 'you', 'my', 'at', 'd', 'its', 'i', 'so', 'was', 'he', 'me', 'no', 'about']

tweet_texts_neg = [ re.sub(r'[^\w\s#@]', '', tweets['text']) for tweets in statuses1 ]
print(tweet_texts_neg[0])
words_neg = [ w for t in tweet_texts_neg for w in t.lower().split() ]
words_neg = list(filter(lambda x: x not in exclude, words_neg))
print(words_neg[1])

shadow dedede update Hammed burger 
dedede


In [142]:
from prettytable import PrettyTable
from collections import Counter

c = Counter(words_neg)
top_words_neg = c.most_common()[:10]
print(top_words_neg) # top 10

pt = PrettyTable(field_names=['Word', 'Count'])
[ pt.add_row(kv) for kv in c.most_common()[:10] ]
pt.align['Word'], pt.align['Count'] = 'l', 'r'
print(pt)

[('burger', 138), ('im', 26), ('burgers', 24), ('want', 21), ('like', 17), ('just', 15), ('fries', 14), ('dont', 13), ('king', 13), ('have', 12)]
+---------+-------+
| Word    | Count |
+---------+-------+
| burger  |   138 |
| im      |    26 |
| burgers |    24 |
| want    |    21 |
| like    |    17 |
| just    |    15 |
| fries   |    14 |
| dont    |    13 |
| king    |    13 |
| have    |    12 |
+---------+-------+


In [143]:
tweet_texts_pos = [ re.sub(r'[^\w\s#@]', '', tweets['text']) for tweets in statuses2 ]
print(tweet_texts_pos[0])
words_pos = [ w for t in tweet_texts_pos for w in t.lower().split() ]
words_pos = list(filter(lambda x: x not in exclude, words_pos))
print(words_pos[1])

@breakcorefan420 @gum_mp3 Hahaha I remember calling in sick the day after getting Arbys MEAT MOUNTAIN p Im scare httpstco5wfWtsuD4M
@gum_mp3


In [144]:
c = Counter(words_pos)
top_words_pos = c.most_common()[:10]
print(top_words_pos) # top 10

pt = PrettyTable(field_names=['Word', 'Count'])
[ pt.add_row(kv) for kv in c.most_common()[:10] ]
pt.align['Word'], pt.align['Count'] = 'l', 'r'
print(pt)

[('burger', 240), ('burgers', 93), ('king', 31), ('have', 30), ('im', 28), ('like', 27), ('good', 27), ('get', 25), ('know', 23), ('will', 22)]
+---------+-------+
| Word    | Count |
+---------+-------+
| burger  |   240 |
| burgers |    93 |
| king    |    31 |
| have    |    30 |
| im      |    28 |
| like    |    27 |
| good    |    27 |
| get     |    25 |
| know    |    23 |
| will    |    22 |
+---------+-------+


In [152]:
# Looking at tweets that have the most used words
for combo in top_words_neg:
    print(combo[0])
    query = { "text": { '$regex': '^.*'+ combo[0] + '.*$', '$options': 'si' } }
    cursor = coll_neg.find(query)
    i = 0
    # Printing first 10 tweets
    for tweet in cursor:
        if i >= 10:
            break
        print(tweet['text'], '\n')
        i += 1

    print("-"*100)

burger
shadow dedede update! Hammed burger :( 

Really hope they bring fries with my burger :( 

what #functional purpose do these serve than #appointing #puppets in their #gizmogazz #glitarati when a burger cost… https://t.co/DgAlPktKtK 

I miss Mooyah.. I miss everything about it! I miss their burgers, their chunky fries, their dense cheese sauce, the… https://t.co/0XsaX0NPzN 

@krikettt88 @DylanGelula I'm not vegan but hard agree on the sadness over the black bean burger loss at restaurants… https://t.co/wnMpoXy9ns 

@DDDunzo @MidKnig22605496 @SeaYou22 @K4TL0_Maiz @TheFunny_mp4 Please don't James Bond McDonald Burger my sister :( 

@Chilly_Pat I actually don’t like any sort of fish :( I’ll try the nugget burger tho 

I couldn’t finish the burger :( https://t.co/BzG51TLtKR 

This is so sad :( I remember being 20 and it absolutely popping at the burger joint right by the FBI building. With… https://t.co/vsuNqSrFiZ 

Accidentally added bacon instead of cheese on my veggie burger :( why

In [153]:
for combo in top_words_pos:
    print(combo[0])
    query = { "text": { '$regex': '^.*'+ combo[0] + '.*$', '$options': 'si' } }
    cursor = coll_pos.find(query)
    i = 0
    # Printing first 10 tweets
    for tweet in cursor:
        if i >= 10:
            break
        print(tweet['text'], '\n')
        i += 1

    print("-"*100)

burger
@allydrawsthings Burger and fries? :D 

Had Steak ‘n Shake for the first time today and was not disappointed :) I love trying burger places, who wants to take me to In ‘n Out next 

@Arceoptryx burger :) 

@absomarvilla You also don’t have a clue on how to spell my name! That said, when it comes to the #impossibleburger… https://t.co/Lcy5F3o6lv 

@acrackedmask @realkagehira Burger first then baby :) 

@Saskgal80 @MarkEllison06 Lol! Well she said she wanted s burger mail and meant burger meal so I didn’t get the fries and she was mad :) 

@radiodreading And now I want a burger, dammit. :) No, I have no clue and I will finish the manga before I dare put… https://t.co/PQuUvMJnp2 

double cheese burger for the first time, i couldn’t finish it :) 

@CheekyVic Just get a non cheese burger 🍔 u are an intelligent girl don't panic!!!!:) 

@freshhmeatz @paradigmnft_ Negative. We only  have Burger King and McDonalds. :D 

--------------------------------------------------------------------

*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 15 minutes' talk) to present about the case study . We will ask *all* teams to present their case studies in class for this case study. 

* **Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through canvas.wpi.edu



# Grading Criteria:

**Totoal Points: 100**


---------------------------------------------------------------------------
**Notebook results:**
    Points: 80


    -----------------------------------
    Question 1:
    Points: 20
    -----------------------------------
    
    -----------------------------------
    Question 2:
    Points: 20
    -----------------------------------
        
    -----------------------------------
    Question 3:
    Points: 20
    -----------------------------------
  
    -----------------------------------
    Question 4:  Business question
    Points: 20
    -----------------------------------


---------------------------------------------------------------------------
**Slides (for 5-10 minutes of presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 
